In [222]:
import os
%load_ext autoreload
%autoreload 2
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/6/7 15:17
# @Author  : Wang Yujia
# @File    : mlp.ipynb
# @Description :
# @TODO:

# 0. what for
1. loss部分逻辑重写，添加`no grad`
2. 添加hooks
3. 使用pad_seq函数取代自己写的pad函数
4. 效果：可以train虽然loss不下降，不会出现NaN的问题

# 1. Preparations
## 1.1 Import

In [1]:
import pandas as pd
import numpy as np
import math
import random
import torch.utils.data
from mydataset import *
import torch.nn as nn
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
# from tensorboardX import SummaryWriter
from visdom import Visdom
from graphviz import Digraph
import torchvision
from torchviz import make_dot
from torch.nn.utils.rnn import pad_sequence
import pygraphviz as pgv
import networkx as nx
import matplotlib.pyplot as plt
import geomloss
import time
from importlib import reload
from functools import partial
import logging
import pytorch_warmup as warmup
logging.basicConfig(filename="loss_info",filemode="w",level=logging.DEBUG)
logger = logging.getLogger(__name__)

import scipy
from torch.nn import KLDivLoss
from torch.autograd.gradcheck import gradcheck
from config import bcolors

import config
import loss
import plot

## 1.2 Global settings and reloading

In [2]:
reload(config)  # 必须reload！！
reload(loss)    # 必须reload！！
reload(plot)
from config import DefaultConfig
from loss import cal_metric
from loss import loss_fn_v2
from loss import loss_fn_wei
from loss import loss_fn_WD
from loss import loss_fn_CE
from loss import validate
from plot import plot_conv_weight
from plot import plot_mu_weight
from plot import plot_pi_weight
from plot import plot_sigma_weight
from plot import plot_net

opt = DefaultConfig()

total_train_step = 0
total_test_step = 0

# Range【一般用不到】
a = 0
b = 255

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# 2. Dataset and Dataloader
1. DataLoader中的`shuffer=True`表示在每一次epoch中都打乱所有数据的顺序，然后以batch为单位从头到尾按顺序取用数据。这样的结果就是不同epoch中的数据都是乱序的,设置随机种子的作用就是让你的每一次训练都乱的一样，

## 2.1 Dataset and spliting


- 设置随机数种子

In [3]:
def setup_seed(seed):
    torch.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    # torch.backends.cudnn.deterministic = True

setup_seed(7)

- 读取data

In [4]:
dataset = myDataset(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.data_key_path, opt.NLL_metric_path)

- 产生index的乱序排列

In [5]:
shuffled_indices = []
DATA_len = 0
# 使用全部的data
if not opt.arr_flag:
    DATA_len = dataset.__len__()
    shuffled_indices = np.random.permutation(DATA_len)

# 使用指定的data
if opt.arr_flag:
    shuffled_indices = np.load(opt.arr_path)
    DATA_len = len(shuffled_indices)
    # np.random.shuffle(shuffled_indices)

train_idx = shuffled_indices[:int(opt.train_pct*DATA_len)]
# train_idx = shuffled_indices
tmp = int((opt.train_pct+opt.vali_pct)*DATA_len)
val_idx = shuffled_indices[int(opt.train_pct*DATA_len):tmp]

test_idx = shuffled_indices[tmp:]
print(train_idx)

[ 807  305  455  939  508  594  835 1082  598 1102   46  761  841  141
  407  334  253  500  734  936  698  446  907 1087 1009  140  463  547
 1155  856   34 1156  703 1121  751  587 1100  509  473  128  788   97
  471  385 1085  525  679 1135  284 1146  186  318 1088 1113    9  457
 1047  451  569 1120 1074  326  377  809  109   98  620 1194  825  828
   83  113  556  674  568  853  351  558   54  656  804 1149  101  344
  851  544  955   40 1021  489  626  664  657  868 1169  151  179 1130
  564  713  743 1014  966  861 1132  146  610  408  662  551  172 1020
  982 1148  431  517  270  858  170  374  816  618  205   17   53 1003
  263  857  716  843  498  228  339  725  752  278  649 1017  108  642
 1195 1174   99  530  632  888  189  961  358 1078  663  757 1051  204
  409  283  562   23  619  216  474  921  950 1162  123  785  769  621
  262  586 1178  541  795   70 1189  396  171  845  168 1125  361  224
  125  706  164  231  264   42  746  872  998  132 1035  430   38  522
  880 

- 根据这个乱序排列抽取dataset

In [8]:
# 保存idx
np.save('shuffled_indices',shuffled_indices)

## 2.2 Dataloader and collating
1. 主要是对label数据进行collate
    - 按照batch中的最大target data长度进行padding，padding with 0
2. 返回的结果多一个batch dim,比如下面的`5`
    - After collating:
        - `torch.Size([5, 3, 300]),torch.Size([5, 87, 2])`
        - `87`是最长的targets data长度

In [ ]:
def test_pad_sequence():
    seq1 = torch.tensor([[ 2., 0.04761905], [3., 0.14285714], [4., 0.04761905]])
    seq2 = torch.tensor([[ 1., 0.04761905]])
    seq3 = torch.tensor([[ 3., 0.14285714], [4., 0.04761905]])
    ls = list((seq1,seq2,seq3))
    ls_length = torch.tensor([3,1,2])
    ans = pad_sequence(ls,batch_first=True)
    print(ans)
    # seq_pak = pack_padded_sequence(ans,ls_length,batch_first=True,enforce_sorted=False)
    # seq_unpacked, lens_unpacked = pad_packed_sequence(seq_pak, batch_first=True)
    # lens_unpacked

### 2.2.1 No rescale

In [6]:
def my_collate_fn(data):
# 这里的data是一个list， list的元素是元组: (self.data, self.label)
# collate_fn的作用是把[(data, label),(data, label)...]转化成([data, data...],[label,label...])
# 假设self.data的一个data的shape为(channels, length), 每一个channel的length相等,
# data[索引到index(batch)][索引到data或者label][索引到channel]

    data_list = []              # training data
    target_metric_list = []     # target data for computing metric of NN, (TARGET=1)
    target_loss_list = []       # target data for computing loss of NN, (TARGET=5 or 1)
    setting_list = []
    metric_list = []

    data_len = len(data)        # 读进来的data batch的大小

    batch = 0

    while batch < data_len:
        # print("shape:",data[batch][0].shape) #shape: (3, 300)

        # 所有GT model
        data_list.append(torch.tensor(data[batch][0]))

        target_metric_list.append(torch.tensor(data[batch][1]))
        target_loss_list.append(torch.tensor(data[batch][2]))
        setting_list.append(torch.tensor(data[batch][3]))
        metric_list.append(torch.tensor(data[batch][4]))
        batch += 1

    # Pad target data with zeros
    target_metric_padded = pad_sequence(target_metric_list,batch_first=True)
    target_loss_padded = pad_sequence(target_loss_list,batch_first=True)
    target_metric_tensor = target_metric_padded.float()
    target_loss_padded = target_loss_padded.float()

    data_tensor = torch.stack(data_list).float()
    setting_tensor = torch.stack(setting_list).float()
    metric_tensor = torch.stack(metric_list).float()

    return data_tensor, target_metric_tensor, target_loss_padded, setting_tensor, metric_tensor

### 2.2.2 Rescale
1. padding 太多0不利于学习，因此把padding的0替换一下：替换成`min(min(data[batch][0][0]),opt.SAFETY)`
    - 替换`min(data[batch][0][0])`/ 一列中最小的prob，但是如果这个值太大，就替换成`opt.SAFETY`
2. 效果: 不全是正向的，效果待定

In [7]:
def my_collate_fn_2(data):
# 这里的data是一个list， list的元素是元组: (self.data, self.label)
# collate_fn的作用是把[(data, label),(data, label)...]转化成([data, data...],[label,label...])
# 假设self.data的一个data的shape为(channels, length), 每一个channel的length相等,
# data[索引到index(batch)][索引到data或者label][索引到channel]

    data_list = []              # training data
    target_metric_list = []     # target data for computing metric of NN, (TARGET=1)
    target_loss_list = []       # target data for computing loss of NN, (TARGET=5)
    setting_list = []
    metric_list = []

    data_len = len(data)        # 读进来的data batch的大小

    batch = 0

    while batch < data_len:
        # print("shape:",data[batch][0].shape) #shape: (3, 300)

        # 对于所有用0填充的data来说，最好用一个数字补齐这些0
        # 补齐方法1:
        data[batch][0][0,np.where(data[batch][0][0]==0)]= min(min(data[batch][0][0]),opt.SAFETY)
        data[batch][0][1,np.where(data[batch][0][1]==0)]= min(min(data[batch][0][1]),opt.SAFETY)

        data_list.append(torch.tensor(data[batch][0]))
        target_metric_list.append(torch.tensor(data[batch][1]))
        target_loss_list.append(torch.tensor(data[batch][2]))
        setting_list.append(torch.tensor(data[batch][3]))
        metric_list.append(torch.tensor(data[batch][4]))
        batch += 1

    # Pad target data with zeros
    target_metric_padded = pad_sequence(target_metric_list,batch_first=True)
    target_loss_padded = pad_sequence(target_loss_list,batch_first=True)
    target_metric_tensor = target_metric_padded.float()
    target_loss_padded = target_loss_padded.float()

    data_tensor = torch.stack(data_list).float()
    setting_tensor = torch.stack(setting_list).float()
    metric_tensor = torch.stack(metric_list).float()

    return data_tensor, target_metric_tensor, target_loss_padded, setting_tensor, metric_tensor

### 2.2.3 Rescale-2
1. padding 太多0不利于学习，

In [367]:
def my_collate_fn_3(data):
# 这里的data是一个list， list的元素是元组: (self.data, self.label)
# collate_fn的作用是把[(data, label),(data, label)...]转化成([data, data...],[label,label...])
# 假设self.data的一个data的shape为(channels, length), 每一个channel的length相等,
# data[索引到index(batch)][索引到data或者label][索引到channel]

    data_list = []              # training data
    target_metric_list = []     # target data for computing metric of NN, (TARGET=1)
    target_loss_list = []       # target data for computing loss of NN, (TARGET=5)
    setting_list = []
    metric_list = []

    data_len = len(data)        # 读进来的data batch的大小

    batch = 0
    # min_pad_0 = np.min(data[:][0][0])
    # min_pad_1 = np.min(data[:][0][1])
    min_pad_0 = np.inf
    min_pad_1 = np.inf

    while batch < data_len:
        # print("shape:",data[batch][0].shape) #shape: (3, 300)
        # # 补齐方法2: batch中的最小值
        # min_pad_0 = min(min_pad_0,min(data[batch][0][0]))
        # min_pad_1 = min(min_pad_1,min(data[batch][0][0]))
        # assert np.min(data[batch][0][0])>=0,"<0!"
        # assert np.min(data[batch][0][1])>=0,"<0!!"

        data_list.append(torch.tensor(data[batch][0]))
        target_metric_list.append(torch.tensor(data[batch][1]))
        target_loss_list.append(torch.tensor(data[batch][2]))
        setting_list.append(torch.tensor(data[batch][3]))
        metric_list.append(torch.tensor(data[batch][4]))
        batch += 1

    # Pad target data with zeros
    target_metric_padded = pad_sequence(target_metric_list,batch_first=True)
    target_loss_padded = pad_sequence(target_loss_list,batch_first=True)
    target_metric_tensor = target_metric_padded.float()
    target_loss_padded = target_loss_padded.float()

    data_tensor = torch.stack(data_list).float()

    data_tensor[data_tensor == 0] = torch.min(data_tensor)
    setting_tensor = torch.stack(setting_list).float()
    metric_tensor = torch.stack(metric_list).float()

    return data_tensor, target_metric_tensor, target_loss_padded, setting_tensor, metric_tensor

### 2.2.4 DataLoader

In [8]:
train_loader = DataLoader(dataset = dataset,batch_size = opt.batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(train_idx), collate_fn = my_collate_fn_2)

val_loader = DataLoader(dataset = dataset,batch_size = opt.batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(val_idx),collate_fn = my_collate_fn_2)

# 注意test_loader的batch size
test_loader = DataLoader(dataset = dataset,batch_size = opt.batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(test_idx),collate_fn = my_collate_fn_2)

# 3. The Net and Init
1. BatchNorm1d: The mean and std are calculated per-dimension over the mini-batches
2.

In [ ]:
### BatchNorm2d测试
def test_BN():
    m = nn.BatchNorm2d(3, affine=False)  # affine: With Learnable Parameters or not
    print('m:', m)
    # The mean and std are calculated per-dimension over the mini-batches
    input = torch.tensor([
        [[1.,2.,3.,4.],[1.,2.,3.,4.],[-1.,-2.,-3.,-4.]],
        [[0.,0.,0.,0.],[0.,0.,0.,0.],[0.,0.,0.,0.]]
    ], requires_grad=True)

    print('input:', input.shape)
    input = input.unsqueeze(dim=2)
    print('input:', input.shape)
    output = m(input) # 归一化
    print('output:', output.shape)
    print('output:', output)

In [ ]:
### 复现batchNorm2d在input shape为3维的情况
input = torch.tensor([[[1.,2.,3.,4.]],[[0.,0.,0.,0.]]])
# print(input.shape)
# torch.mean(input),torch.var(input,unbiased = False)
(input-torch.mean(input))/ torch.sqrt(torch.var(input,unbiased = False))

## 3.1 Init, Printer, Hook
- 设置网络初始权重: 不太work

In [204]:

class model_param_init(nn.Module):
    def __init__(self, model):
        super().__init__()
        assert isinstance(model, nn.Module), 'model not a class nn.Module'
        self.net = model
        self.initParam()

    def forward(self):
        for param in self.net.parameters():
            # nn.init.zeros_(param)
            # nn.init.ones_(param)
            # nn.init.normal_(param, mean=0, std=1)
            # nn.init.uniform_(param, a=0, b=1)
            # nn.init.constant_(param, val=1)   # 将所有权重初始化为1
            # nn.init.eye_(param)  # 只能将二维的tensor初始化为单位矩阵
            # nn.init.xavier_uniform_(param, gain=1)  # Glorot初始化  得到的张量是从-a——a中采用的
            # print(param)  x.unsqueeze(0)
            nn.init.xavier_normal_(param.unsqueeze(0), gain=1)   # 得到的张量是从0-std采样的
            # nn.init.kaiming_normal_(param, a=0, mode='fan_in', nonlinearity='relu') # he初始化方法
            # nn.init.kaiming_uniform_(param)

- print网络每层结构

In [ ]:
DEBUG = False
class PrintLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        if(DEBUG):
            print("This layer: ")
            print(x)      #print(x.shape)
        return x

- hook_backward_fn: 输入端的grad，输出端的grad，这里称呼的**输入与输出是站在前向传播的角度的**。如果模块有多个输入与输出的话， 其`grad_input`和`grad_output`可以是tuple类型。
- 与forward不同的是，backward传播的时候，**不仅反向传递input和output的grad，还会传递模块Parameter的grad**：
    - 比如fc模块，其`grad_input`是一个三元组的tuple，（对bias的梯度，对输入的梯度，对w的梯度）；
    - conv模块`grad_input`也是一个三元组tuple，为（对输入的梯度，对w的梯度，对bias的梯度

In [ ]:
# hook functions have to take these 3 input
def hook_forward_fn(module, input, output):
    print("It's forward: ")
    print(f"module: {module}")
    print(f"input: {input}")
    print(f"output: {output}")
    print("="*20)

def hook_backward_fn(module, grad_input, grad_output):
    print("It's backward: ")
    print(f"module: {module}")
    print(f"grad_input: {grad_input}")
    print(f"grad_output: {grad_output}")
    print("="*20)

def hook_backward_fn_pi(module, grad_input, grad_output):
    print("It's backward in pi: ")
    print(f"module: {module}")
    print(f"grad_input: {grad_input}")
    print(f"grad_output: {grad_output}")
    print("="*20)

def hook_backward_fn_sigma(module, grad_input, grad_output):
    print("It's backward in sigma: ")
    print(f"module: {module}")
    print(f"grad_input: {grad_input}")
    print(f"grad_output: {grad_output}")
    print("="*20)

def hook_backward_fn_mu(module, grad_input, grad_output):
    print("It's backward in mu: ")
    print(f"module: {module}")
    print(f"grad_input: {grad_input}")
    print(f"grad_output: {grad_output}")
    print("="*20)

## 3.2 Conv结构-1
### 3.2.1 1层conv: 1=>1
1.  Conv=>BN=>AC

In [9]:
# 1569
class Conv_block_1(nn.Module):
    def __init__(self, ch_out=1,kernel_size=9, stride=3) -> None:
        super().__init__()

        self.kernel_size = (3,kernel_size)
        self.stride = (3,stride)
        self.ln_in = int((300-self.kernel_size[1])/self.stride[1]+1)

        self.ac_func = nn.Softplus()

        self.conv = nn.Conv2d(in_channels=1, out_channels=ch_out, kernel_size=self.kernel_size, stride=self.stride, padding=0,bias=True)
        self.BN_aff1 = nn.BatchNorm1d(num_features=1,affine=True)
        self.BN_aff2 = nn.BatchNorm1d(num_features=self.ln_in,affine=True)      # works better

    def forward(self, x):
        # Conv=>BN=>AC
        x = self.conv(x)
        # 方法一：
        # x = torch.squeeze(x,dim=2)
        # x = self.ac_func(self.BN_aff1(x))

        # 方法二：works better
        x = torch.squeeze(x)
        x = self.ac_func(self.BN_aff2(x))

        return x

class Conv_1_1(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians, ch_out=1, kernel_size=9, stride=3) -> None:
        super().__init__()

        self.kernel_size = (3,kernel_size)
        self.stride = (3,stride)
        self.ln_in = int((300-self.kernel_size[1])/self.stride[1]+1)

        self.BN1 = nn.BatchNorm1d(num_features=3,affine=True)

        self.layer_pi = Conv_block_1(ch_out=1,kernel_size=kernel_size,stride=stride)
        self.layer_mu = Conv_block_1(ch_out=1,kernel_size=kernel_size,stride=stride)
        self.layer_sigma = Conv_block_1(ch_out=1,kernel_size=kernel_size,stride=stride)

        self.ac_func = nn.Softplus()

        self.z_pi = nn.Sequential(
            nn.Linear(self.ln_in, n_gaussians),
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(self.ln_in, n_gaussians)
        self.z_sigma = nn.Linear(self.ln_in, n_gaussians)


    def forward(self, x):

        x = self.BN1(x)
        x = torch.unsqueeze(x,dim=1)                     # torch.Size([B, 1, 3, 300])

        x_pi = torch.squeeze(self.layer_pi(x))          # 不加squeeze不行
        x_mu = torch.squeeze(self.layer_mu(x))
        x_sigma = torch.squeeze(self.layer_sigma(x))

        pi = self.z_pi(x_pi)
        mu = self.z_mu(x_mu)
        sigma = self.z_sigma(x_sigma)

        sigma = torch.exp(sigma)
        sigma = torch.clamp(sigma,1e-4)

        # return x1_3
        return pi, mu, sigma

In [10]:
mlp = Conv_1_1(opt.N_gaussians)
mlp = mlp.to(device=device)
summary(mlp, (3,300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1               [-1, 3, 300]               6
            Conv2d-2             [-1, 1, 1, 98]              28
       BatchNorm1d-3                   [-1, 98]             196
          Softplus-4                   [-1, 98]               0
      Conv_block_1-5                   [-1, 98]               0
            Conv2d-6             [-1, 1, 1, 98]              28
       BatchNorm1d-7                   [-1, 98]             196
          Softplus-8                   [-1, 98]               0
      Conv_block_1-9                   [-1, 98]               0
           Conv2d-10             [-1, 1, 1, 98]              28
      BatchNorm1d-11                   [-1, 98]             196
         Softplus-12                   [-1, 98]               0
     Conv_block_1-13                   [-1, 98]               0
           Linear-14                   

### 3.2.2 2层conv: 1=>1=>1
1. 比上面多了一个conv1d的提取
2. 做BN有2个思路：
    - 第一个conv结束，有91个features，BN之后接另一个Conv提取这些不同features里的信息，目的是最后得到mu等的特征
    - 第一个conv结束，有1个features，BN之后接另一个Conv提取/压缩这1个features里的信息，目的是最后得到mu等的特征

In [1293]:
# 1569
class Conv_block_2(nn.Module):
    def __init__(self, ch_out=1,kernel_size=12, stride=3) -> None:
        super().__init__()

        self.kernel_size1 = (3,kernel_size)
        self.stride1 = (3,stride)
        self.ln_in = int((300-self.kernel_size1[1])/self.stride1[1]+1)

        self.ac_func = nn.Softplus()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=self.kernel_size1, stride=self.stride1, padding=0,bias=True)
        # self.BN_aff1 = nn.BatchNorm1d(num_features=1,affine=True)
        self.BN_aff1 = nn.BatchNorm1d(num_features=self.ln_in,affine=True)      # works better

        self.kernel_size2 = int(kernel_size/2)     # 可以调这个
        self.stride2 = stride
        self.ln_in2 = int((self.ln_in-self.kernel_size2)/self.stride2+1)
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=ch_out, kernel_size=self.kernel_size2, stride=self.stride2, padding=0,bias=True)
        self.BN_aff2 = nn.BatchNorm1d(num_features=self.ln_in2,affine=True)      # works better


    def forward(self, x):
        # Conv=>BN=>AC
        x = self.conv1(x)
        # 方法二：works better
        x = torch.squeeze(x)
        x = self.ac_func(self.BN_aff1(x))

        x = torch.unsqueeze(x,dim=1)

        x = self.conv2(x)
        x = torch.squeeze(x)
        x = self.ac_func(self.BN_aff2(x))

        return x

class Conv_1_2(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians, ch_out=1, kernel_size=12, stride=3) -> None:
        super().__init__()

        self.kernel_size = (3,kernel_size)
        self.stride = (3,stride)
        # self.ln_in = int((300-self.kernel_size[1])/self.stride[1]+1)
        self.ln_in = 31

        self.BN1 = nn.BatchNorm1d(num_features=3,affine=True)

        self.layer_pi = Conv_block_2(ch_out=1,kernel_size=kernel_size,stride=stride)
        self.layer_mu = Conv_block_2(ch_out=1,kernel_size=kernel_size,stride=stride)
        self.layer_sigma = Conv_block_2(ch_out=1,kernel_size=kernel_size,stride=stride)

        self.ac_func = nn.Softplus()

        self.z_pi = nn.Sequential(
            nn.Linear(self.ln_in, n_gaussians),
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(self.ln_in, n_gaussians)
        self.z_sigma = nn.Linear(self.ln_in, n_gaussians)

    def forward(self, x):

        x = self.BN1(x)
        x = torch.unsqueeze(x,dim=1)                     # torch.Size([B, 1, 3, 300])

        x_pi = torch.squeeze(self.layer_pi(x))          # 不加squeeze不行
        x_mu = torch.squeeze(self.layer_pi(x))
        x_sigma = torch.squeeze(self.layer_pi(x))

        pi = self.z_pi(x_pi)

        # mu = self.z_mu(x_mu)
        # mu = self.ac_func(self.z_mu(x_mu))
        mu = self.z_mu(x_mu)
        sigma = self.z_sigma(x_sigma)

        sigma = torch.exp(sigma)
        sigma = torch.clamp(sigma,1e-4)

        # return x_pi
        return pi, mu, sigma

In [1294]:
mlp = Conv_1_2(opt.N_gaussians)
mlp = mlp.to(device=device)
summary(mlp, (3,300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1               [-1, 3, 300]               6
            Conv2d-2             [-1, 1, 1, 97]              37
       BatchNorm1d-3                   [-1, 97]             194
          Softplus-4                   [-1, 97]               0
            Conv1d-5                [-1, 1, 31]               7
       BatchNorm1d-6                   [-1, 31]              62
          Softplus-7                   [-1, 31]               0
      Conv_block_2-8                   [-1, 31]               0
            Conv2d-9             [-1, 1, 1, 97]              37
      BatchNorm1d-10                   [-1, 97]             194
         Softplus-11                   [-1, 97]               0
           Conv1d-12                [-1, 1, 31]               7
      BatchNorm1d-13                   [-1, 31]              62
         Softplus-14                   

### 3.2.3 2层conv: 1=>2=>1
1. Conv2d+squeeze+conv1d

In [1273]:
# 573
class Conv_block_3(nn.Module):
    def __init__(self, ch_out=2,kernel_size=12, stride=3) -> None:
        super().__init__()

        self.kernel_size1 = (3,kernel_size)
        self.stride1 = (3,stride)
        self.ln_in = int((300-self.kernel_size1[1])/self.stride1[1]+1)

        self.ac_func = nn.Softplus()
        # [B, 1, 3, 300] => [B, 2, 1, ln_in]
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=ch_out, kernel_size=self.kernel_size1, stride=self.stride1, padding=0,bias=True)
        self.BN_aff1 = nn.BatchNorm1d(num_features=ch_out,affine=True)


        self.kernel_size2 = int(kernel_size/2)
        self.stride2 = stride
        self.conv2 = nn.Conv1d(in_channels=ch_out, out_channels=1, kernel_size=self.kernel_size2, stride=self.stride2, padding=0,bias=True)
        self.BN_aff2 = nn.BatchNorm1d(num_features=31,affine=True)

    def forward(self, x):
        # Conv=>BN=>AC
        x = self.conv1(x)
        x = torch.squeeze(x,dim=2)  # [2, 2, 1, 97]
        x = self.ac_func(self.BN_aff1(x))

        x = torch.squeeze(x)
        # [B, 2, 97] => [B, 1, 31]
        x = self.conv2(x)
        x = torch.squeeze(x)       # [B, 1, 31] => [B, 31]
        x = self.ac_func(self.BN_aff2(x))

        return x

class Conv_1_3(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians, ch_out=2, kernel_size=12, stride=3) -> None:
        super().__init__()

        self.kernel_size = (3,kernel_size)
        self.stride = (3,stride)
        # self.ln_in_1 = int((300-self.kernel_size[1])/self.stride[1]+1)
        self.ln_in = 31

        self.BN1 = nn.BatchNorm1d(num_features=3,affine=True)

        self.layer_pi = Conv_block_3(ch_out=ch_out,kernel_size=kernel_size,stride=stride)
        self.layer_mu = Conv_block_3(ch_out=ch_out,kernel_size=kernel_size,stride=stride)
        self.layer_sigma = Conv_block_3(ch_out=ch_out,kernel_size=kernel_size,stride=stride)

        self.ac_func = nn.Softplus()

        self.z_pi = nn.Sequential(
            nn.Linear(self.ln_in, n_gaussians),
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(self.ln_in, n_gaussians)
        self.z_sigma = nn.Linear(self.ln_in, n_gaussians)

    def forward(self, x):

        x = self.BN1(x)
        x = torch.unsqueeze(x,dim=1)                     # torch.Size([B, 1, 3, 300])

        x_pi = torch.squeeze(self.layer_pi(x))          # 不加squeeze不行
        x_mu = torch.squeeze(self.layer_pi(x))
        x_sigma = torch.squeeze(self.layer_pi(x))

        pi = self.z_pi(x_pi)

        # mu = self.z_mu(x_mu)
        mu = self.ac_func(self.z_mu(x_mu))
        sigma = self.z_sigma(x_sigma)

        sigma = torch.exp(sigma)
        sigma = torch.clamp(sigma,1e-4)

        # return x_pi
        return pi, mu, sigma

In [1256]:
mlp = Conv_1_3(opt.N_gaussians)
mlp = mlp.to(device=device)
summary(mlp, (3,300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1               [-1, 3, 300]               6
            Conv2d-2             [-1, 2, 1, 97]              74
       BatchNorm1d-3                [-1, 2, 97]               4
          Softplus-4                [-1, 2, 97]               0
            Conv1d-5                [-1, 1, 31]              13
       BatchNorm1d-6                   [-1, 31]              62
          Softplus-7                   [-1, 31]               0
      Conv_block_2-8                   [-1, 31]               0
            Conv2d-9             [-1, 2, 1, 97]              74
      BatchNorm1d-10                [-1, 2, 97]               4
         Softplus-11                [-1, 2, 97]               0
           Conv1d-12                [-1, 1, 31]              13
      BatchNorm1d-13                   [-1, 31]              62
         Softplus-14                   

### 3.2.4 weibull
1.

In [15]:
# 1569
class Conv_block_4(nn.Module):
    def __init__(self, ch_out=1,kernel_size=12, stride=3) -> None:
        super().__init__()

        self.kernel_size = (3,kernel_size)
        self.stride = (3,stride)
        self.ln_in = int((300-self.kernel_size[1])/self.stride[1]+1)

        self.ac_func = nn.Softplus()

        self.conv = nn.Conv2d(in_channels=1, out_channels=ch_out, kernel_size=self.kernel_size, stride=self.stride, padding=0,bias=True)
        self.BN_aff1 = nn.BatchNorm1d(num_features=1,affine=True)
        self.BN_aff2 = nn.BatchNorm1d(num_features=self.ln_in,affine=True)      # works better

    def forward(self, x):
        # Conv=>BN=>AC
        x = self.conv(x)
        # 方法一：
        # x = torch.squeeze(x,dim=2)
        # x = self.ac_func(self.BN_aff1(x))

        # 方法二：works better
        x = torch.squeeze(x)
        x = self.ac_func(self.BN_aff2(x))

        return x

class Conv_1_4(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians, ch_out=1, kernel_size=12, stride=3) -> None:
        super().__init__()

        self.kernel_size = (3,kernel_size)
        self.stride = (3,stride)
        self.ln_in = int((300-self.kernel_size[1])/self.stride[1]+1)

        self.BN1 = nn.BatchNorm1d(num_features=3,affine=True)

        self.layer_pi = Conv_block_4(ch_out=1,kernel_size=kernel_size,stride=stride)
        self.layer_scale = Conv_block_4(ch_out=1,kernel_size=kernel_size,stride=stride)
        self.layer_shape = Conv_block_4(ch_out=1,kernel_size=kernel_size,stride=stride)

        self.ac_func = nn.Softplus()

        self.z_pi = nn.Sequential(
            nn.Linear(self.ln_in, n_gaussians),
            nn.Softmax(dim=1)
        )
        self.z_scale = nn.Linear(self.ln_in, n_gaussians)
        self.z_shape = nn.Linear(self.ln_in, n_gaussians)

    def forward(self, x):

        x = self.BN1(x)
        x = torch.unsqueeze(x,dim=1)                     # torch.Size([B, 1, 3, 300])

        x_pi = torch.squeeze(self.layer_pi(x))          # 不加squeeze不行
        x_scale = torch.squeeze(self.layer_scale(x))
        x_shape = torch.squeeze(self.layer_shape(x))

        pi = self.z_pi(x_pi)
        scale = torch.exp(self.z_scale(x_scale))
        scale = torch.clamp(scale,1e-4)
        shape = torch.exp(self.z_shape(x_shape))
        shape = torch.clamp(shape,1e-4)

        return pi,scale,shape

In [1546]:
mlp = Conv_1_4(opt.N_gaussians)
mlp = mlp.to(device=device)
summary(mlp, (3,300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1               [-1, 3, 300]               6
            Conv2d-2             [-1, 1, 1, 97]              37
       BatchNorm1d-3                   [-1, 97]             194
          Softplus-4                   [-1, 97]               0
      Conv_block_4-5                   [-1, 97]               0
            Conv2d-6             [-1, 1, 1, 97]              37
       BatchNorm1d-7                   [-1, 97]             194
          Softplus-8                   [-1, 97]               0
      Conv_block_4-9                   [-1, 97]               0
           Conv2d-10             [-1, 1, 1, 97]              37
      BatchNorm1d-11                   [-1, 97]             194
         Softplus-12                   [-1, 97]               0
     Conv_block_4-13                   [-1, 97]               0
           Linear-14                   

## 3.3 Conv结构-2
1. 换了一下conv的结构

In [11]:
# 参数量747
class MLP_1_2(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=3,affine=False)
        self.BN2 = nn.BatchNorm2d(num_features=3,affine=True)
        self.BN3 = nn.BatchNorm2d(num_features=6,affine=True)
        self.drop = nn.Dropout(0.3)

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=(1,6), stride=(1,3), padding=0,bias=False)
        self.ac_func = nn.Softplus()
        # self.ac_func = nn.ReLU()

        # 3, 3, 99
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(3,6), stride=6, padding=0,bias=False)
        # 6,1,33

        self.conv3 = nn.Conv2d(in_channels=6, out_channels=6, kernel_size=(1,3), stride=3, padding=0,bias=False)

        self.flatten = nn.Flatten()
        #self.linear1 = nn.Linear(48, 18)

        #无clip
        self.z_pi = nn.Sequential(
            nn.Linear(30, n_gaussians),
            nn.Softmax(dim=1)
        )

        # 有clip
        # self.z_pi = nn.Linear(30, n_gaussians)
        self.z_mu = nn.Linear(30, n_gaussians)
        self.z_sigma = nn.Linear(30, n_gaussians)

    def forward(self, x):
        x = self.BN1(x)
        # 加一个height维度

        x = torch.unsqueeze(x,dim=1)
        x = self.conv1(x)
        x = self.BN2(x)
        x = self.ac_func(self.drop(x))

        x = self.conv2(x)
        x = self.BN3(x)
        x = self.ac_func(self.drop(x))

        # x = torch.unsqueeze(x,dim=2)
        x = self.conv3(x)
        x = self.ac_func(self.drop(x))

        x = self.flatten(x)
        # x = self.linear1(x)
        # x = self.ac_func(self.drop(x))
        mu = self.z_mu(x)
        pi = self.z_pi(x)

        sigma = torch.exp(self.z_sigma(x))
        sigma = torch.clamp(sigma,1e-4)

        # return x
        return pi,mu,sigma

## 3.4 MLP结构
### 3.4.1 4层

In [12]:
# 参数量304,000+
# 4层
class MLP_1_3(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=3,affine=True)
        self.BN2 = nn.BatchNorm1d(num_features=300,affine=True)
        self.BN3 = nn.BatchNorm1d(num_features=100,affine=True)
        self.BN4 = nn.BatchNorm1d(num_features=30,affine=True)
        self.BN5 = nn.BatchNorm1d(num_features=12,affine=True)
        self.BN6 = nn.BatchNorm1d(num_features=12,affine=True)
        self.drop = nn.Dropout(0.3)

        self.linear1 = nn.Linear(900, 300)
        self.linear2 = nn.Linear(300, 100)
        self.linear3 = nn.Linear(100, 30)
        self.linear4 = nn.Linear(30, 12)

        self.ac_func = nn.Softplus()
        self.ac_func2 = nn.SELU()
        self.ac_func3 = nn.LeakyReLU()

        self.flatten = nn.Flatten()

        self.z_pi = nn.Sequential(
            nn.Linear(12, n_gaussians),  # 30个params要learn
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(12, n_gaussians)
        self.z_sigma = nn.Linear(12, n_gaussians)

        # SELU Init
        # for m in self.modules():
        #     if isinstance(m, (nn.Linear)):
        #         nn.init.kaiming_normal_(m.weight, nonlinearity='linear')

    def forward(self, x):
        x = self.BN1(x)
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.BN2(x)
        x = self.ac_func3(self.drop(x))

        x = self.linear2(x)
        x = self.BN3(x)
        x = self.ac_func3(self.drop(x))

        x = self.linear3(x)
        x = self.BN4(x)
        x = self.ac_func3(self.drop(x))

        x = self.linear4(x)
        x = self.BN5(x)
        x = self.ac_func3(self.drop(x))

        x = self.BN6(x)
        pi = self.z_pi(x)
        # pi = self.z_pi(torch.clamp(x,1e-3))
        mu = self.z_mu(x)
        # sigma = torch.exp(self.z_sigma(x))
        sigma = F.elu(self.z_sigma(x)) + 1
        sigma = torch.clamp(sigma,1e-4)

        # return x
        return pi, mu, sigma

### 3.4.2 2层

In [1660]:
# 参数量91,000+
# 2层MLP+MDN
class MLP_1_4(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=3,affine=True)
        self.BN2 = nn.BatchNorm1d(num_features=100,affine=True)
        self.BN3 = nn.BatchNorm1d(num_features=12,affine=True)
        self.drop = nn.Dropout(0.3)

        self.linear1 = nn.Linear(900, 100)
        self.linear2 = nn.Linear(100, 12)

        self.ac_func = nn.Softplus()
        self.ac_func2 = nn.SELU()
        self.ac_func3 = nn.LeakyReLU()

        self.flatten = nn.Flatten()

        self.z_pi = nn.Sequential(
            nn.Linear(12, n_gaussians),  # 30个params要learn
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(12, n_gaussians)
        self.z_sigma = nn.Linear(12, n_gaussians)


    def forward(self, x):
        # print("x.shape: ",x.shape)  # torch.Size([40, 3, 300])
        x = self.BN1(x)
        x = self.flatten(x)

        x = self.linear1(x)
        # x = self.BN2(x)
        x = self.ac_func3(self.drop(x))

        x = self.linear2(x)
        # x = self.BN3(x)
        x = self.ac_func3(self.drop(x))

        # 连续的ac？
        pi = self.z_pi(x)
        # mu = self.z_mu(x)

        # # sigma = torch.exp(self.z_sigma(x))
        # # sigma = torch.exp(self.ac_func(self.z_sigma(x)))
        # sigma = torch.exp(self.z_sigma(x))
        # # sigma = F.elu(self.z_sigma(x)) + 1
        # sigma = torch.clamp(sigma,1e-4)

        scale = torch.exp(self.z_mu(x))
        scale = torch.clamp(scale,1e-4)
        shape = torch.exp(self.z_sigma(x))
        shape = torch.clamp(shape,1e-4)

        # return pi, mu, sigma
        return pi, scale, shape

### 3.4.4 1层+avgpool

In [238]:
# 参数量2823
# 1层MLP+MDN
class MLP_1_5(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=3,affine=True)
        self.BN2 = nn.BatchNorm1d(num_features=9,affine=True)
        self.drop = nn.Dropout(0.3)

        self.avgpool = nn.AvgPool2d(kernel_size=(1,3))  # kernel_size=(1,Channel)
        self.linear1 = nn.Linear(300, 9)

        self.ac_func = nn.Softplus()

        self.flatten = nn.Flatten()

        self.z_pi = nn.Sequential(
            nn.Linear(9, n_gaussians),  # 30个params要learn
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(9, n_gaussians)
        self.z_sigma = nn.Linear(9, n_gaussians)

        # SELU Init
        # for m in self.modules():
        #     if isinstance(m, (nn.Linear)):
        #         nn.init.kaiming_normal_(m.weight, nonlinearity='linear')

        # 初始化权重
        for m in self.modules():
            if isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                # m.weight.data.normal_(0, 0.02)
                # nn.init.xavier_normal_(m.weight,gain=1)
                nn.init.orthogonal_(m.weight)
                nn.init.zeros_(m.bias)
        # nn.init.xavier_normal_(self.z_mu.weight,gain=10)

    def forward(self, x):
        x = self.BN1(x)
        # print("before transpose: ",x.shape)
        x = torch.transpose(x, 1, 2)        # [B,N,C]
        # print("before avgpool: ",x.shape)
        x = self.avgpool(x)
        # print("after avgpool: ",x.shape)
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.BN2(x)
        x = self.ac_func(self.drop(x))

        pi = self.z_pi(x)
        # pi = self.z_pi(torch.clamp(x,1e-3))
        mu = self.z_mu(x)

        sigma = torch.exp(self.z_sigma(x))
        # sigma = F.elu(self.z_sigma(x)) + 1
        sigma = torch.clamp(sigma,1e-4)

        return pi, mu, sigma

In [218]:
def test_pool():
    # torch.Size([40, 3, 300])
    a = torch.tensor([[[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5]]])
    # print(a.shape)
    a = torch.transpose(a, 1, 2)
    print(a)
    ans = F.avg_pool2d(a,(1,3))
    print(ans.shape)    # [B,N,C]
# test_pool()

### 3.4.5 1层+无avgpool

In [249]:
# 参数量8233
# 1层MLP+MDN
class MLP_1_6(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=3,affine=True)
        self.BN2 = nn.BatchNorm1d(num_features=9,affine=True)
        self.drop = nn.Dropout(0.3)

        self.linear1 = nn.Linear(900, 9)

        self.ac_func = nn.Softplus()

        self.flatten = nn.Flatten()

        self.z_pi = nn.Sequential(
            nn.Linear(9, n_gaussians),  # 30个params要learn
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(9, n_gaussians)
        self.z_sigma = nn.Linear(9, n_gaussians)

        # SELU Init
        # for m in self.modules():
        #     if isinstance(m, (nn.Linear)):
        #         nn.init.kaiming_normal_(m.weight, nonlinearity='linear')

        # for m in self.modules():
        #     if isinstance(m, nn.BatchNorm1d):
        #         m.weight.data.fill_(1)
        #         m.bias.data.zero_()
        #     elif isinstance(m, nn.Linear):
        #         # m.weight.data.normal_(0, 0.02)
        #         # nn.init.xavier_normal_(m.weight,gain=1)
        #         nn.init.orthogonal_(m.weight)
        #         nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.BN1(x)
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.BN2(x)
        x = self.ac_func(self.drop(x))

        pi = self.z_pi(x)
        # pi = self.z_pi(torch.clamp(x,1e-3))
        mu = self.z_mu(x)

        sigma = torch.exp(self.z_sigma(x))
        # sigma = F.elu(self.z_sigma(x)) + 1
        sigma = torch.clamp(sigma,1e-4)

        return pi, mu, sigma

## 3.5 (3,100)的结构
1. 适配(3,100)的training data长度而不是(3,300)
### 3.5.1

In [ ]:
# # 参数量：228
# class MLP_2_2(nn.Module):
#     # code->generate->override methods
#     def __init__(self, n_gaussians) -> None:
#         super().__init__()
#         self.BN = nn.BatchNorm2d(num_features=3,affine=False)
#         self.BN_aff = nn.BatchNorm2d(num_features=3,affine=True)
#         self.drop = nn.Dropout(0.2)
#
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,3), stride=(1,3), padding=0,bias=False)
#         # self.ac_func = nn.Softplus()
#         self.ac_func = nn.LeakyReLU()
#         self.ac_func2 = nn.ReLU()
#
#         # self.ac_func = nn.Sigmoid()
#         # self.ac_func = nn.Tanh()
#
#         self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,2), stride=(1,2), padding=0,bias=False)
#
#         self.conv3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,3), stride=(1,3), padding=0,bias=False)
#
#         self.flatten = nn.Flatten()
#
#         self.z_pi = nn.Sequential(
#             nn.Linear(15, n_gaussians),
#             nn.Softmax(dim=1)
#         )
#
#         self.z_mu = nn.Linear(15, n_gaussians)
#         self.z_sigma = nn.Linear(15, n_gaussians)
#
#     def forward(self, x):
#         # 加一个height维度
#         x = torch.unsqueeze(x,dim=2)
#         x = self.BN(x)
#         x = self.conv1(x)
#         x = self.ac_func(self.drop(x))
#
#         x = self.BN_aff(x)
#         x = self.conv2(x)
#         x = self.ac_func(self.drop(x))
#
#         x = self.BN_aff(x)
#         x = self.conv3(x)
#         x = self.ac_func(self.drop(x))
#         #
#         x = self.flatten(x)
#
#         pi = self.z_pi(torch.clamp(x,1e-3))
#         # pi = self.z_pi(x)
#         mu = self.z_mu(x)
#         sigma = torch.exp(self.z_sigma(x))
#
#         # return x
#         return pi, mu, sigma

- MLP结构+长度=100

### 3.5.2 MLP-2层

In [81]:
# 参数量 18699
class MLP_2_3(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=3,affine=True)
        self.BN2 = nn.BatchNorm1d(num_features=60,affine=True)
        self.BN3 = nn.BatchNorm1d(num_features=9,affine=True)
        self.drop = nn.Dropout(0.5)

        self.linear1 = nn.Linear(300, 60)
        self.linear2 = nn.Linear(60, 9)

        self.ac_func = nn.Softplus()

        self.flatten = nn.Flatten()

        self.z_pi = nn.Sequential(
            nn.Linear(9, n_gaussians),  # 30个params要learn
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(9, n_gaussians)
        self.z_sigma = nn.Linear(9, n_gaussians)

    def forward(self, x):
        x = self.BN1(x)
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.BN2(x)
        x = self.ac_func(self.drop(x))

        x = self.linear2(x)
        x = self.BN3(x)
        x = self.ac_func(self.drop(x))

        pi = self.z_pi(x)
        # pi = self.z_pi(torch.clamp(x,1e-3))
        mu = self.z_mu(x)
        sigma = torch.exp(self.z_sigma(x))
        sigma = torch.clamp(sigma,1e-4)

        # return x
        return pi, mu, sigma

### 3.5.3 MLP-1层

In [135]:
# 参数量2823
class MLP_2_4(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=3,affine=True)
        self.BN2 = nn.BatchNorm1d(num_features=9,affine=True)
        self.drop = nn.Dropout(0.5)

        self.linear1 = nn.Linear(300, 9)

        self.ac_func = nn.Softplus()

        self.flatten = nn.Flatten()

        self.z_pi = nn.Sequential(
            nn.Linear(9, n_gaussians),  # 30个params要learn
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(9, n_gaussians)
        self.z_sigma = nn.Linear(9, n_gaussians)

    def forward(self, x):
        x = self.BN1(x)
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.BN2(x)
        x = self.ac_func(self.drop(x))

        pi = self.z_pi(x)
        # pi = self.z_pi(torch.clamp(x,1e-3))
        mu = self.z_mu(x)
        sigma = torch.exp(self.z_sigma(x))
        sigma = torch.clamp(sigma,1e-4)

        # return x
        return pi, mu, sigma

## 3.6 (3,60)的结构
1. 适配(3,60)的training data长度而不是(3,300)

In [ ]:
# # 参数量：174
# class MLP_3_1(nn.Module):
#     # code->generate->override methods
#     def __init__(self, n_gaussians) -> None:
#         super().__init__()
#         self.BN = nn.BatchNorm2d(num_features=3,affine=False)
#         self.BN_aff = nn.BatchNorm2d(num_features=3,affine=True)
#         self.drop = nn.Dropout(0.2)
#
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,3), stride=(1,3), padding=0,bias=False)
#         # self.ac_func = nn.Softplus()
#         self.ac_func = nn.LeakyReLU()
#         self.ac_func2 = nn.ReLU()
#
#         # self.ac_func = nn.Sigmoid()
#         # self.ac_func = nn.Tanh()
#
#         self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,2), stride=(1,2), padding=0,bias=False)
#
#         self.conv3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,3), stride=(1,3), padding=0,bias=False)
#
#         self.flatten = nn.Flatten()
#         self.linear = nn.Linear(24, 9)
#
#         self.z_pi = nn.Sequential(
#             nn.Linear(9, n_gaussians),
#             nn.Softmax(dim=1)
#         )
#
#         self.z_mu = nn.Linear(9, n_gaussians)
#         self.z_sigma = nn.Linear(9, n_gaussians)
#
#     def forward(self, x):
#         # 加一个height维度
#         x = torch.unsqueeze(x,dim=2)
#         x = self.BN(x)
#         x = self.conv1(x)
#         x = self.ac_func(self.drop(x))      # 3, 1, 20
#
#         x = self.BN_aff(x)
#         x = self.conv2(x)
#         x = self.ac_func(self.drop(x))      # 3, 1, 10
#
#         x = self.BN_aff(x)
#         x = self.conv3(x)
#         x = self.ac_func(self.drop(x))      # 3, 1, 8
#
#         x = self.flatten(x)
#         # print(x.shape)
#         # x = self.linear(x)
#         #
#         pi = self.z_pi(x)
#         mu = self.z_mu(x)
#         sigma = torch.exp(self.z_sigma(x))
#
#         # return x
#         return pi, mu, sigma

- MLP

In [ ]:
# # 参数量：5799
# class MLP_3_2(nn.Module):
#     # code->generate->override methods
#     def __init__(self, n_gaussians) -> None:
#         super().__init__()
#         self.BN = nn.BatchNorm2d(num_features=3,affine=False)
#         self.BN_aff = nn.BatchNorm2d(num_features=3,affine=True)
#         self.drop = nn.Dropout(0.2)
#
#         self.linear1 = nn.Linear(180,30)
#         self.linear2 = nn.Linear(30,9)
#         # self.ac_func = nn.Softplus()
#         self.ac_func = nn.LeakyReLU()
#         self.ac_func2 = nn.ReLU()
#
#         # self.ac_func = nn.Sigmoid()
#         # self.ac_func = nn.Tanh()
#
#         self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,2), stride=(1,2), padding=0,bias=False)
#
#         self.conv3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,3), stride=(1,3), padding=0,bias=False)
#
#         self.flatten = nn.Flatten()
#         self.linear = nn.Linear(24, 9)
#
#         self.z_pi = nn.Sequential(
#             nn.Linear(9, n_gaussians),
#             nn.Softmax(dim=1)
#         )
#
#         self.z_mu = nn.Linear(9, n_gaussians)
#         self.z_sigma = nn.Linear(9, n_gaussians)
#
#     def forward(self, x):
#         # 加一个height维度
#         x = torch.unsqueeze(x,dim=2)
#         x = self.BN(x)
#         x = self.flatten(x)
#
#         x = self.linear1(x)
#         x = self.ac_func(self.drop(x))
#
#         x = self.linear2(x)
#         x = self.ac_func(self.drop(x))
#
#         pi = self.z_pi(x)
#         mu = self.z_mu(x)
#         sigma = torch.exp(self.z_sigma(x))
#
#         # return x
#         return pi, mu, sigma

# 4. The Loss

## 4.0 The metric
1. 以NLL作为metric
2. 理论模型的metric已经提前算好，读进来.
3. 注意这里的metric比较的是target还是target_5:
    - 建议使用target data而不是target_5，因为后者是为了方便NN的learning，前者才是真正的比较NLL
    - 使用前者也要用cdf进行比较吧。。比如GT的4实际上是我们这里的[3.5,4.5]？是高斯分布在[a-0.5,a+0.5]上的cdf作为a的“单点分布”

In [48]:
# from loss import cal_metric

## 4.1 NLL loss
### 4.1.1 original version

In [42]:
# from loss import loss_fn_v2

### 4.1.2 version 3 [LogSumExp]
1. $ exponent_i = \log_{}{\alpha_i } -1/2*\log_{}{2\pi } -\log_{}{\sigma_i }- \frac{(x-\mu_i )^2}{2\sigma_i^2} $
2. $ NLL =-\sum_{j}^{N_S}LogSumExp= -\sum_{j}^{N_S} \log\sum_{i}^{N_G} \exp \{exponent_i\}$
3. 不稳定！lr不要设超过1e-2；而且loss会变成负值。。最开始x_max是负值，后面会变成正值 which means prob值开始>1了

In [ ]:
from loss import log_sum_exp
from loss import loss_fn_v3

## 4.2 cdf loss
1. 注意把targets data的路径改一下

In [21]:
# cdf loss
from loss import loss_fn_cdf

## 4.3 交叉熵/ KL
1. 自己写的，没有用torch的函数

In [ ]:
from loss import loss_fn_CE

In [ ]:
def loss_test():
    loss_2 = torch.tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
    loss_3 = loss_2[torch.nonzero(loss_2)].view(-1,1)         # 去掉所有的log(0)
    print("loss_3:",loss_3.shape)

    MIN_LOSS = torch.min(loss_2) if torch.min(loss_2)>0 else 1e-20

    print("MIN_LOSS：",MIN_LOSS)
    # 用MIN_loss替代loss_2为0的部分
    loss_2[loss_2==0] = MIN_LOSS


    loss_padded = [MIN_LOSS]* (loss_2.shape[0]-loss_3.shape[0])
    loss_padded_ts = torch.tensor(loss_padded) # ,device=device
    loss_3 = torch.cat((loss_3[:,0],loss_padded_ts))
    loss_3.shape, torch.log(loss_3)
    print("data: ",loss_3.data)
    print("data: ",type(loss_3))
# loss_test()

## 4.4 W-distance
1. 拿到target data的两列，按照第一列去predict
2. Ref: https://www.kernel-operations.io/geomloss/api/pytorch-api.html

# 5 Plot
1. draw:
    - loss: training data的loss和test data的loss趋势
    - MLP的网络结构（.png）
    - target distrb.和pred. distrb.
    - 所有target data的分布图


In [11]:
viz = Visdom(env="001")

## 5.1 draw 所有的target data
1. 实际上在`i < 50`里决定画前50 or what target data

In [ ]:
def draw_all_target_data():
    # target data
    # target_path = r"../data/targets"
    target_path = r"../data/targets_5"
    # target_path里有全部的target data地址
    target_all_path = os.listdir(target_path)
    len_target = len(target_all_path)
    print(f"一共有 *{len_target}* 组 target data")

    # Init
    # viz_env_str = "target_bar"
    # viz_env_str = "target_line"
    # viz_env_str = "target_hist"
    viz_env_str = "target_scatter"
    viz = Visdom(env = viz_env_str)

    for i in range(len_target):
        target_path_i_path = os.path.join(target_path,target_all_path[i])
        target_df = pd.read_csv(target_path_i_path,encoding="utf-8")

        # Init
        win_str = str(target_all_path[i])
        title_str = "Target Distrb. of "+win_str

        if(i < 50):
            # Hist plot which is not that great
            if viz_env_str == "target_hist" :
                viz.histogram(X = np.array(target_df.N), env=viz_env_str, win=win_str,
                            opts= dict(title=title_str,numbins = 50))

            # Line plot
            if viz_env_str == "target_line" :
                target_df.drop_duplicates(inplace=True)
                viz.line(X = np.array(target_df.N),Y= np.array(target_df.P), env=viz_env_str, win=win_str,opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

            # Bar plot. 这个看起来是最准确的一个
            if viz_env_str == "target_bar" :
                target_df.drop_duplicates(inplace=True)
                arr_str = [str(i) for i in np.array(target_df.N)]
                viz.bar(X=np.array(target_df.P), env=viz_env_str, win=win_str,
                        opts=dict(title=title_str,rownames=arr_str,stacked = False))

            # Scatter plot
            if viz_env_str == "target_scatter" :
                target_df.drop_duplicates(inplace=True)
                viz.scatter(X=np.array(target_df),env=viz_env_str, win=win_str,opts=dict(title=title_str,markersize = 3))
    print("Done")

# draw_all_target_data()

- 画target data的cdf

In [ ]:
def draw_all_target_data_cdf():
    # target data
    # target_path = r"../data/targets"
    target_path = r"../data/targets_5_cdf"
    # target_path里有全部的target data地址
    target_all_path = os.listdir(target_path)
    len_target = len(target_all_path)
    print(f"一共有 *{len_target}* 组 target data")

    # Init，只需要改这个str就ok
    # viz_env_str = "target_barCDF"
    # viz_env_str = "target_lineCDF"
    # viz_env_str = "target_histCDF"
    viz_env_str = "target_scatterCDF"
    viz = Visdom(env = viz_env_str)

    for i in range(len_target):
        target_path_i_path = os.path.join(target_path,target_all_path[i])
        target_df = pd.read_csv(target_path_i_path,encoding="utf-8")

        # Init
        win_str = str(target_all_path[i])
        title_str = "Target Distrb. of "+win_str

        if(i < 50):
            # Hist plot which is not that great
            if viz_env_str == "target_histCDF" :
                viz.histogram(X = np.array(target_df.N), env=viz_env_str, win=win_str,
                            opts= dict(title=title_str,numbins = 50))

            # Line plot
            if viz_env_str == "target_lineCDF" :
                target_df.drop_duplicates(inplace=True)
                viz.line(X = np.array(target_df.N),Y= np.array(target_df.P), env=viz_env_str, win=win_str,opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

            # Bar plot. 这个看起来是最准确的一个
            if viz_env_str == "target_barCDF" :
                target_df.drop_duplicates(inplace=True)
                arr_str = [str(i) for i in np.array(target_df.N)]
                viz.bar(X=np.array(target_df.P), env=viz_env_str, win=win_str,
                        opts=dict(title=title_str,rownames=arr_str,stacked = False))

            # Scatter plot
            if viz_env_str == "target_scatterCDF" :
                target_df.drop_duplicates(inplace=True)
                viz.scatter(X=np.array(target_df),env=viz_env_str, win=win_str,opts=dict(title=title_str,markersize = 3))
    print("Done")

# draw_all_target_data_cdf()

- 也是画target data，但是是平滑处理
- `savgol_filter`参数解析:
    - y：代表曲线点坐标（x,y）中的y
    - window_length：窗口长度，该值需为正奇整数。值越小，曲线越贴近真实曲线；值越大，平滑效果越厉害
    - polyorder: 对窗口内的数据点进行k阶多项式拟合，k的值需要小于window_length。值越大，曲线越贴近真实曲线；值越小，平滑效果越厉害
    - mode：确定了要应用滤波器的填充信号的扩展类型。（This determines the type of extension to use for the padded signal to which the filter is applied. ）
- **实际上没用到这个函数**

In [ ]:
def draw_smooth_target_data():
    # target data
    target_path = r"../data/targets"
    # target_path里有全部的target data地址
    target_all_path = os.listdir(target_path)
    len_target = len(target_all_path)
    print(f"一共有 *{len_target}* 组 target data")

    # Init
    viz_env_str = "target_bar"
    # viz_env_str = "target_distrb"
    # viz_env_str = "target_hist"
    # viz_env_str = "target_scatter"
    viz = Visdom(env = viz_env_str)

    for i in range(len_target):
        target_path_i_path = os.path.join(target_path,target_all_path[i])
        target_df = pd.read_csv(target_path_i_path,encoding="utf-8")

        # Init
        win_str = str(target_all_path[i])
        title_str = "Target Distrb. of "+win_str

        if(i < 100):
            # Smoothen the data
            y_smooth = scipy.signal.savgol_filter(target_df.P,window_length=53,polyorder=3)
            # y_smooth = scipy.signal.savgol_filter(target_df.P, 99, 1, mode= 'nearest')
            target_df.P = y_smooth

            # Hist plot which is not that great
            # viz.histogram(X = np.array(target_df.N), env=viz_env_str, win=win_str,
            #             opts= dict(title=title_str,numbins = 50))

            # Line plot
            # target_df.drop_duplicates(inplace=True)
            # viz.line(X = np.array(target_df.N),Y= np.array(target_df.P), env=viz_env_str, win=win_str,opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

            # Bar plot. 这个看起来是最准确的一个
            target_df.drop_duplicates(inplace=True)
            arr_str = [str(i) for i in np.array(target_df.N)]
            viz.bar(X=np.array(target_df.P), env=viz_env_str, win=win_str,
                    opts=dict(title=title_str,rownames=arr_str))

            # Scatter plot
            # target_df.drop_duplicates(inplace=True)
            # viz.scatter(X=np.array(target_df),env=viz_env_str, win=win_str,opts=dict(title=title_str,markersize = 3))
    print("Done")

# draw_smooth_target_data()

In [ ]:
#### Test for drawing
def test_draw():
    viz = Visdom(env="001")

    mu = torch.tensor([0,10,20])
    sigma = torch.tensor([1,1,1])
    duration = torch.tensor([0,1,2,0])
    duration = torch.repeat_interleave(duration.unsqueeze(dim=1), repeats=3, dim=1)
    m = torch.distributions.Normal(loc=mu, scale=sigma)
    pi = torch.tensor([0.2,0.3,0.5])

    # draw
    x_0 = torch.tensor(np.arange(0,1000))
    x = torch.repeat_interleave(x_0.unsqueeze(dim=1), repeats=3, dim=1)
    y = torch.exp(m.log_prob(x))
    y_sum = torch.unsqueeze(torch.sum(pi*y,dim=1),dim=1)
    viz.line(X = x_0,Y= torch.cat([y,y_sum],dim = 1), env="001", win="test_draw_2",
            opts= dict(title='test_draw', legend=['N1', 'N2', 'N3','NNN']))
# test_draw()

## 5.2 plot mdn

### 5.2.1 mdn pdf的图像[with input data]
1. 求解[1,6,11...]上的cdf：
    - 求解[1,6,11..]的cdf，再求解[6,11,..]，两者相减

In [ ]:
# smoothed_y[t] = average(y[t-k], y[t-k+1], ..., y[t+k-1], y[t+k])
# sm表示滑动窗口大小,为2*k+1,
def smooth(data, sm=1):
    if sm > 1:
        smooth_data = []
        for d in data:
            y = np.ones(sm)*1.0/sm
            d = np.convolve(y, d, "same")

            smooth_data.append(d)

    return smooth_data

In [ ]:
def draw_mdn_2(pi, mu, sigma, target, input, total_train_step, N_gaussians):
    """
    画两条曲线，pred-1表示归一化前，pred-2表示归一化后

    :param pi:
    :param mu:
    :param sigma:
    :param target:
    :param input:
    :param total_train_step:
    :param opt.N_gaussians:
    :return:
    """
    # 只画一个batch中的第一个

    # The target distrb.
    target = torch.unique(target,dim=0)    # drop the duplicate
    n_target = target[:,0]
    non_zero_idx = torch.nonzero(n_target)
    n = n_target[non_zero_idx]

    p_target = target[:,1]
    p = p_target[non_zero_idx]
    max_n = int(max(n).item())

    # The predicted distrb.
    m = torch.distributions.Normal(mu,sigma)          # Gaussian
    # m = torch.distributions.Laplace(mu,sigma)           # Laplace

    x_0 = torch.arange(1,(max_n+opt.TARGET),opt.TARGET).to(device=device)

    x = torch.repeat_interleave(x_0.unsqueeze(dim=1), repeats=opt.N_gaussians, dim=1)   # expand dim
    # 求解每个长度为TARGET的区间上的cdf
    y = (m.cdf(x+opt.TARGET) - m.cdf(x)).to(device=device)

    # 方法一：
    y_pred_1 = torch.sum(pi*y,dim=1)
    # 方法二：做一下(0,1)上的归一化
    y_pred_2 = y_pred_1/y_pred_1.sum()

    # The input distrb.
    input_data = input[0:2,:]
    x_input = torch.arange(1,(300+1),opt.SCALE).to(device=device)
    y_input_0 = (input_data[0,:]).to(device=device)
    y_input_1 = (input_data[1,:]).to(device=device)

    # Init
    win_str = "total_train_step = "+str(total_train_step)
    title_str = "Distrb. in "+win_str
    viz.line(X = [0.],Y = [0.], env="001", win=win_str, opts= dict(title=title_str))

    # Visdom本身不能把hist和line画在一个window中
    # 如果想画一起只能是两条line
    # Plot y_target
    # viz.histogram(X = n, env="001", win=win_str,
    #         opts= dict(title=title_str))
    viz.line(X = n,Y= p, env="001", win=win_str, update="append", name='target',
            opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

    # Plot y_pred
    viz.line(X = x_0,Y= y_pred_1, env="001", win=win_str, update="append", name='pred-1',
            opts= dict(title=title_str))

    # Plot y_pred_2
    viz.line(X = x_0,Y= y_pred_2, env="001", win=win_str, update="append", name='pred-2',
            opts= dict(title=title_str))

    # Plot y_input
    # 如果input data长度短，全部画完
    if(len(x_input) <= max_n):
        viz.line(X = x_input,Y= y_input_0, env="001", win=win_str, update="append", name='GT-1', opts= dict(title=title_str))
        viz.line(X = x_input,Y= y_input_1, env="001", win=win_str, update="append", name='GT-2', opts= dict(title=title_str))
    # 如果input data长度长，则截断
    else:
        x_input_0 = x_input[0:max_n]
        y_input_2 = y_input_0[0:max_n]/y_input_0[0:max_n].sum()
        y_input_3 = y_input_1[0:max_n]/y_input_1[0:max_n].sum()
        viz.line(X = x_input_0,Y= y_input_2, env="001", win=win_str, update="append", name='GT-1', opts= dict(title=title_str))
        viz.line(X = x_input_0,Y= y_input_3, env="001", win=win_str, update="append", name='GT-2', opts= dict(title=title_str))

### 5.2.2 MDN+weibull[with input data]

In [1840]:
def draw_mdn_wei(pi, scale, shape, target, input, total_train_step, N_gaussians):
    """
    画两条曲线，pred-1表示归一化前，pred-2表示归一化后

    :param pi:
    :param scale:
    :param shape:
    :param target: target data records
    :param input:  input data of NN / GT pred. 但是不能画这个！！这个是为了input做了归一化的
    :param total_train_step:
    :param opt.N_gaussians:
    :return:
    """
    # 只画一个batch中的第一个

    # The target distrb.
    target = torch.unique(target,dim=0)    # drop the duplicate
    n_target = target[:,0]
    non_zero_idx = torch.nonzero(n_target)
    n = n_target[non_zero_idx]

    p_target = target[:,1]
    p = p_target[non_zero_idx]
    max_n = int(max(n).item())

    # The predicted distrb.
    m = torch.distributions.Weibull(scale, shape)

    x_0 = torch.arange(1,(max_n+1)).to(device=device)
    x = torch.repeat_interleave(x_0.unsqueeze(dim=1), repeats=N_gaussians, dim=1)   # expand dim
    # 求解每个小区间上的cdf作为likelihood
    y = (m.cdf(x+0.5) - m.cdf(x-0.5)).to(device=device)

    # 方法一：
    y_pred_1 = torch.sum(pi*y,dim=1)
    # 方法二：做一下(0,1)上的归一化
    y_pred_2 = y_pred_1/y_pred_1.sum()

    # The input distrb.
    input_data = input[0:2,:]
    x_input = torch.arange(1,(300+1),opt.SCALE).to(device=device)
    y_input_0 = (input_data[0,:]).to(device=device)
    y_input_1 = (input_data[1,:]).to(device=device)

    # Init
    win_str = "total_train_step = "+str(total_train_step)
    title_str = "Distrb. in "+win_str
    viz.line(X = [0.],Y = [0.], env="001", win=win_str, opts= dict(title=title_str))

    # Visdom本身不能把hist和line画在一个window中
    # 如果想画一起只能是两条line
    # Plot y_target
    # viz.histogram(X = n, env="001", win=win_str,
    #         opts= dict(title=title_str))
    viz.line(X = n,Y= p, env="001", win=win_str, update="append", name='target',
            opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

    # Plot y_pred
    viz.line(X = x_0,Y= y_pred_1, env="001", win=win_str, update="append", name='pred-1',
            opts= dict(title=title_str))

    # Plot y_pred_2
    viz.line(X = x_0,Y= y_pred_2, env="001", win=win_str, update="append", name='pred-2',
            opts= dict(title=title_str))

    # # Plot y_input
    # # 如果input data长度短，全部画完
    # if(len(x_input) <= max_n):
    #     viz.line(X = x_input,Y= y_input_0, env="001", win=win_str, update="append", name='GT-1', opts= dict(title=title_str))
    #     viz.line(X = x_input,Y= y_input_1, env="001", win=win_str, update="append", name='GT-2', opts= dict(title=title_str))
    # # 如果input data长度长，则截断
    # else:
    #     x_input_0 = x_input[0:max_n]
    #     y_input_2 = y_input_0[0:max_n]/y_input_0[0:max_n].sum()
    #     y_input_3 = y_input_1[0:max_n]/y_input_1[0:max_n].sum()
    #     viz.line(X = x_input_0,Y= y_input_2, env="001", win=win_str, update="append", name='GT-1', opts= dict(title=title_str))
    #     viz.line(X = x_input_0,Y= y_input_3, env="001", win=win_str, update="append", name='GT-2', opts= dict(title=title_str))

## 5.3 png格式的net structure

In [ ]:
def draw_the_net_png():

    x = torch.randn([1, 3, 300])  # 定义网络的输入值
    mlp = MLP_1_1(opt.N_gaussians)
    y = mlp(x)                    # 获取网络的预测值

    MyConvNetVis = make_dot(y, params=dict(list(mlp.named_parameters()) + [('x', x)]))
    MyConvNetVis.format = "png"
    # 指定文件生成的文件夹
    MyConvNetVis.directory = "data_pic"
    # 生成文件
    MyConvNetVis.view()
# draw_the_net_png()

## 5.4 plot the loss
1. 直接根据参数决定draw什么loss，反正loss都画在一个window里面

In [12]:
win_train_loss_str = "The Loss of BATCH in the Training Data"
win_vali_loss_str = "The Loss in the Vali Data"
win_train_epoch_loss_str = "The Loss of EPOCH in the Training Data"

def draw_loss(X_step, loss, win_str):
    viz.line(X = [X_step], Y = [loss],win=win_str, update="append",
        opts= dict(title=win_str))


## 5.5 plot the metric

In [13]:
win_vali_metric_str = "The NLL of ALL vali data"
def draw_metric(X_step, total_vali_metric, GT_metric):

    viz.line(X = [X_step], Y = [[total_vali_metric,GT_metric[0,0],GT_metric[0,1]]],win=win_vali_metric_str, update="append", opts= dict(title=win_vali_metric_str, legend=['pred','GT-1','GT-2'], showlegend=True,xlabel="epoch", ylabel="NLL"))


## 5.6 grad graph
https://github.com/t-vi/pytorch-tvmisc/blob/master/visualize/bad_grad_viz.ipynb

In [595]:

def iter_graph(root, callback):
    queue = [root]
    seen = set()
    while queue:
        # 队头出fn然后判断fn是否seen
        fn = queue.pop()
        if fn in seen:
            continue
        # unseen加入queue，并且继续沿着fn向下递归（DFS？）
        seen.add(fn)
        for next_fn, _ in fn.next_functions:
            # Alert：只有非none的才会加入queue，然后在register_grad里被fn_dict记录下grad_input
            if next_fn is not None:
                queue.append(next_fn)
        callback(fn)

def register_hooks(var):
    fn_dict = {}  # 记录了grad名字和值（grad_input）
    def hook_cb(fn):
        def register_grad(grad_input, grad_output):
            fn_dict[fn] = grad_input
        fn.register_hook(register_grad)

    # 1. 递归注册grad_input
    iter_graph(var.grad_fn, hook_cb)

    # def is_bad_grad(grad_output):
    #     if grad_output is None:
    #         return False
    #     return grad_output.isnan().any() or (grad_output.abs() >= 1e6).any()
    def is_bad_grad(grad_output):
        if grad_output is None:
                return True
        grad_output = grad_output.data
        return grad_output.ne(grad_output).any() or grad_output.gt(1e6).any()

    # 2. graph已经从grad input构建完了，还有grad output的判断以及颜色的选择
    def make_dot():
        node_attr = dict(style='filled',
                        shape='box',
                        align='left',
                        fontsize='12',
                        ranksep='0.1',
                        height='0.2')
        dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))

        def size_to_str(size):
            return '('+(', ').join(map(str, size))+')'

        def build_graph(fn):
            if hasattr(fn, 'variable'):  # if GradAccumulator
                u = fn.variable
                node_name = 'Variable\n ' + size_to_str(u.size())
                dot.node(str(id(u)), node_name, fillcolor='lightblue')
            else:  # 不是variable属性，白色
                assert fn in fn_dict, fn   # 判断fn在不在fn_dict里面！
                fillcolor = 'white'
                if any(is_bad_grad(gi) for gi in fn_dict[fn]):
                    fillcolor = 'red'
                dot.node(str(id(fn)), str(type(fn).__name__), fillcolor=fillcolor)
            for next_fn, _ in fn.next_functions:
                if next_fn is not None:
                    next_id = id(getattr(next_fn, 'variable', next_fn))
                    dot.edge(str(next_id), str(id(fn)))
        iter_graph(var.grad_fn, build_graph)

        return dot

    return make_dot

# x = torch.randn(10, 10, requires_grad=True)
# y = torch.randn(10, 10, requires_grad=True)
#
# z = x / (y * 0)
# z = z.sum() * 2
# get_dot = register_hooks(z)


# with torch.autograd.detect_anomaly():
#     z.backward()
#     dot = get_dot()
#     dot.save('tmp.dot') # to get .dot
#     dot.render('tmp') # to get SVG
#     # dot # in Jupyter, you can just render the variable

# 6 Training
## 6.1 Init

In [16]:
# torch.autograd.set_detect_anomaly(True)
# mlp = MLP_1_4(opt.N_gaussians)
# (9,3)不错
mlp = Conv_1_4(opt.N_gaussians,kernel_size=9,stride=3)
# mlp = Conv_1_1(opt.N_gaussians,kernel_size=12,stride=3)
# mlp = Conv_1_2(opt.N_gaussians)

# Init the params
# mlp = model_param_init(mlp)

# Save the init params
# torch.save(mlp.state_dict(), 'mlp_init.pth')

# Load the saved model
# model_path_WD = "mlp_train=300_WD.pth"
# model_path_init = 'mlp_init.pth'
# model_path_LSE = "mlp_train=300_LSE.pth"
# model_path_MLE = "mlp_train=300_MLE.pth"
#
# model_data = torch.load(model_path_init)
# mlp.load_state_dict(model_data)

mlp = mlp.to(device=device)
summary(mlp, (3,300))

############ learning rate strategy ############
# 1.
# Set different lr for params
# id: id() 函数返回对象的唯一标识符，标识符是一个整数。返回对象的内存地址。
shape_params = list(map(id, mlp.z_shape.parameters()))
scale_params = list(map(id, mlp.z_scale.parameters()))
pi_params = list(map(id, mlp.z_pi.parameters()))


params_id = shape_params + scale_params + pi_params

base_params = filter(lambda p: id(p) not in params_id, mlp.parameters())
params = [{'params': base_params},         # 如果对某个参数不指定学习率，就使用最外层的默认学习率
          {'params': mlp.z_pi.parameters(), 'lr': opt.lr_for_pi},
        {'params': mlp.z_shape.parameters(), 'lr': opt.learning_rate},
          {'params': mlp.z_scale.parameters(), 'lr': opt.lr_for_scale}]

optimizer = torch.optim.AdamW(params, lr=opt.learning_rate,weight_decay=opt.weight_decay)

# warmup_scheduler = warmup.ExponentialWarmup(optimizer, warmup_period=10)
# 2.
# Or set lr decay
# StepLR为步进，每step_size个epoch，lr*gamma
scheduler  = torch.optim.lr_scheduler.StepLR(optimizer,step_size=opt.StepLR_step_size,gamma=0.9)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 70, eta_min = 1e-7)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.98, last_epoch=-1)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5, verbose=False, threshold=1e-3, threshold_mode='abs', cooldown=0, min_lr=1e-7, eps=1e-7)
# threshold:只关注超过阈值的显著变化；cooldown：触发一次条件后，等待一定epoch再进行检测，避免lr下降过速；

# # Set the hooks
# #mlp.conv1.register_forward_hook(hook_forward_fn)
# mlp.conv1.register_full_backward_hook(hook_backward_fn)
#
# #mlp.linear1.register_forward_hook(hook_forward_fn)
# #mlp.linear1.register_full_backward_hook(hook_backward_fn)
#
# #mlp.z_pi.register_forward_hook(hook_forward_fn)
# mlp.z_pi.register_full_backward_hook(hook_backward_fn_pi)
#
# #mlp.z_mu.register_forward_hook(hook_forward_fn)
# mlp.z_mu.register_full_backward_hook(hook_backward_fn_mu)
#
# mlp.z_sigma.register_forward_hook(hook_forward_fn)
# mlp.z_sigma.register_full_[2, 3, 300]backward_hook(hook_backward_fn_sigma)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1               [-1, 3, 300]               6
            Conv2d-2             [-1, 1, 1, 98]              28
       BatchNorm1d-3                   [-1, 98]             196
          Softplus-4                   [-1, 98]               0
      Conv_block_4-5                   [-1, 98]               0
            Conv2d-6             [-1, 1, 1, 98]              28
       BatchNorm1d-7                   [-1, 98]             196
          Softplus-8                   [-1, 98]               0
      Conv_block_4-9                   [-1, 98]               0
           Conv2d-10             [-1, 1, 1, 98]              28
      BatchNorm1d-11                   [-1, 98]             196
         Softplus-12                   [-1, 98]               0
     Conv_block_4-13                   [-1, 98]               0
           Linear-14                   

In [17]:
writer = SummaryWriter(log_dir="logs-MLP/"+opt.logs_str,flush_secs=60)

# Init the vis win
viz.line(X = [0.],Y = [0.], env="001", win=win_train_loss_str, opts= dict(title=win_train_loss_str))
viz.line(X = [0.],Y = [0.], env="001", win=win_train_epoch_loss_str, opts= dict(title=win_train_epoch_loss_str))
viz.line(X = [0.],Y = [[0.,0.,0.]], env="001", win=win_vali_metric_str, opts= dict(title=win_vali_metric_str,legend=['pred','GT-1','GT-2'], showlegend=True,xlabel="epoch", ylabel="NLL"))

mlp.eval()
with torch.no_grad():
    total_vali_metric, GT_metric = validate(mlp,val_loader,opt.N_gaussians, opt.MIN_LOSS,device)
    draw_metric(0, total_vali_metric.cpu(), GT_metric)
plot_net(writer,mlp,torch.randn((2,3,300),device=device))
print("Done")

Done


## 6.2 grad check


In [18]:
# # grad check
# input_test = torch.randn((2,3,300), requires_grad=True,device=device)
# test_ans = gradcheck(mlp.to(device), input_test, eps=1e-3)  #, eps=1e-6, atol=1e-4
# print("Are the gradients correct: ", test_ans)

## 6.3 START HERE


In [19]:
# filename = "../log_file.txt"
# f = open(filename,'w')
total_train_step = 0
EPOCH_NUM = 70

train_start_time = time.time()
mlp.train()

for epoch in range(EPOCH_NUM):
    # # Save the net structure
    # net_file_name = "net_before_epoch"+str(epoch)+".pth"
    # net_path = opt.net_root_path + net_file_name
    # torch.save(mlp.state_dict(), net_path)

    epoch_train_loss = 0
    # plot_conv(writer,mlp,epoch)
    print(f"========== Now this is EPOCH {epoch} ,lr is {scheduler.get_last_lr()}==========")
    for batch_id,data in enumerate(train_loader):

        input_data, target_plot, target_loss, setting, _ = data
        # print(f"---- {batch_id} batch----")
        # Do the inference
        input_data = input_data.to(device)
        target_loss = target_loss.to(device)
        target_plot = target_plot.to(device)
        pi, mu, sigma = mlp(input_data)

        # Save the params
        # params = list(mlp.named_parameters())

        # Cal the MLE loss and draw the distrb.
        # loss = loss_fn_v2(pi, mu, sigma, target_loss, opt.N_gaussians,opt.TARGET, opt.SAFETY, device)
        loss = loss_fn_wei(pi, mu, sigma, target_loss, opt.N_gaussians,opt.TARGET, opt.SAFETY, device)
        # loss = loss_fn_v3(pi, mu, sigma, target_loss, opt.N_gaussians,device)
        # loss = loss_fn_v5(pi, mu, sigma, target_loss, opt.N_gaussians,device)

        # Cal the CDF loss and draw the distrb.
        # loss = loss_fn_cdf(pi, mu, sigma, target, opt.N_gaussians)

        # Cal the CE or KL loss and draw the distrb.
        # loss  = loss_fn_CE(pi, mu, sigma, target_loss, opt.N_gaussians,opt.TARGET, opt.SAFETY, device)

        # Cal the WD loss and draw the distrb.
        # loss  = loss_fn_WD(pi, mu, sigma, target_loss, opt.N_gaussians,opt.TARGET,device)

        epoch_train_loss += loss.item()
        draw_loss(total_train_step, loss.item(),win_train_loss_str)

        # Optim
        optimizer.zero_grad()
        loss.backward()

        ########### before step() ###############

        # for name, parms in mlp.named_parameters():
        #     print("=====before step()=====")
        #     print('-->name:', name)
        #     print('-->params:', parms.data)
        #     # print('-->grad_requirs:',parms.requires_grad)
        #     print('-->grad_value:',parms.grad)
        #     print("===")

        ######

        # 反向传播时检测是否有异常值，定位code
        # with torch.autograd.detect_anomaly():
        #     loss.backward()

        ######
        # get_dot = register_hooks(loss)
        # dot = get_dot()
        # dot.save('tmp.dot')   # to get .dot
        # dot.render('tmp')     # to get pdf

        ######
        # Print grad check
        # v_n = []    # name
        # v_v = []    # value
        # v_g = []    # grad
        # for name, parameter in mlp.named_parameters():
        #     v_n.append(name)
        #     v_v.append(parameter.detach().cpu().numpy() if parameter is not None else [0])
        #     v_g.append(parameter.grad.detach().cpu().numpy() if parameter.grad is not None else [0])
        # for i in range(len(v_n)):
        #     # if np.max(v_v[i]).item() - np.min(v_v[i]).item() < 1e-6:
        #     #     color = bcolors.FAIL + '*'
        #     if np.isnan(v_g[i]).any() or np.isnan(v_v[i]).any():
        #         color = bcolors.FAIL + '*'
        #     else:
        #         color = bcolors.OKGREEN + ' '
        #     print('%svalue %s: %.3e ~ %.3e' % (color, v_n[i], np.min(v_v[i]).item(), np.max(v_v[i]).item()))
        #     print('%sgrad  %s: %.3e ~ %.3e' % (color, v_n[i], np.min(v_g[i]).item(), np.max(v_g[i]).item()))

        ######
        optimizer.step()

        # for name, parms in mlp.named_parameters():
        #     print("=====After step()=====")
        #     print('-->name:', name)
        #     print('-->params:', parms.data)
        #     # print('-->grad_requirs:',parms.requires_grad)
        #     print('-->grad_value:',parms.grad)
        #     print("===")

        # Only draw the 1st result in a training batch (5 in total)
        # if total_train_step % 20 == 0:
        #     with torch.no_grad():
        #         # draw_mdn_2(pi[0,:].clone().detach(), mu[0,:].clone().detach(), sigma[0,:].clone().detach(), target_plot[0].clone().detach(), input_data[0].clone().detach(),total_train_step, opt.N_gaussians)
        #         draw_mdn_wei(pi[0,:].clone().detach(), mu[0,:].clone().detach(), sigma[0,:].clone().detach(), target_plot[0].clone().detach(), input_data[0].clone().detach(),total_train_step, opt.N_gaussians)
        #         # draw_mdn_cdf(pi[0,:].detach(), m[0], target[0].detach(), total_train_step, loss_list[0],opt.N_gaussians)
        total_train_step += 1

    ########### Do validation

    mlp.eval()
    with torch.no_grad():
        total_vali_metric, GT_metric = validate(mlp,val_loader,opt.N_gaussians, opt.MIN_LOSS,device)
        draw_metric(epoch+1, total_vali_metric.cpu(), GT_metric)
        writer.add_scalars("metric/"+opt.tag_str,{"pred":total_vali_metric.cpu(),
                                     "GT-1":GT_metric[0,0],
                                     "GT-2":GT_metric[0,1]},epoch)
    mlp.train()

    # Plot the loss in this EPOCH
    print(f"========== IN EPOCH {epoch} the total loss is {epoch_train_loss} ==========")
    print(f"========== IN EPOCH {epoch} the vali NLL loss is {total_vali_metric.detach().cpu().numpy()} ==========")
    print(f"========== IN EPOCH {epoch} the GT metric is {GT_metric.detach().cpu().numpy()} ==========")

    # # Save the net structure
    # net_file_name = "net_after_epoch"+str(epoch)+".pth"
    # net_path = opt.net_root_path + net_file_name
    # torch.save(mlp.state_dict(), net_path)

    # Plot loss of this EPOCH
    draw_loss(epoch, epoch_train_loss,win_train_epoch_loss_str)
    # writer.add_scalars("Loss of EPOCH",{"Train":epoch_train_loss},epoch)
    # scheduler.step(total_vali_metric)
    scheduler.step()

    if epoch%3==0:
        # Record the weight
        plot_conv_weight(writer,mlp,epoch,opt.tag_str)
        plot_mu_weight(writer,mlp,epoch,opt.tag_str)
        plot_pi_weight(writer,mlp,epoch,opt.tag_str)
        plot_sigma_weight(writer,mlp,epoch,opt.tag_str)

# f.close()
train_end_time = time.time()
print(f"Total training time when epoch= *{EPOCH_NUM}* is *{train_end_time-train_start_time} *s")
writer.close()
print("Done")

========== Now this is EPOCH 0 ,lr is [0.005, 0.001, 0.005, 0.001]==========
========== IN EPOCH 0 the total loss is 34222.83172607422 ==========
========== IN EPOCH 0 the vali NLL loss is 8.253518104553223 ==========
========== IN EPOCH 0 the GT metric is [[ 6.568752 10.179958]] ==========
========== Now this is EPOCH 1 ,lr is [0.005, 0.001, 0.005, 0.001]==========
========== IN EPOCH 1 the total loss is 17902.85125732422 ==========
========== IN EPOCH 1 the vali NLL loss is 7.874137878417969 ==========
========== IN EPOCH 1 the GT metric is [[ 6.568752 10.179958]] ==========
========== Now this is EPOCH 2 ,lr is [0.005, 0.001, 0.005, 0.001]==========
========== IN EPOCH 2 the total loss is 16732.331329345703 ==========
========== IN EPOCH 2 the vali NLL loss is 7.429294109344482 ==========
========== IN EPOCH 2 the GT metric is [[ 6.5687523 10.179958 ]] ==========
========== Now this is EPOCH 3 ,lr is [0.005, 0.001, 0.005, 0.001]==========
========== IN EPOCH 3 the total loss is 1510

- save

In [20]:
# model_path_MLE = "mlp_train=300_MLE.pth"
model_path_MLP = "mlp_train=300_MLP.pth"
model_path_Conv = "mlp_train=300_Conv.pth"
model_path_LSE = "mlp_train=300_LSE.pth"
torch.save(mlp.state_dict(), model_path_Conv)
# torch.save(mlp.state_dict(), model_path_MLE)

# viz.delete_env("001_test")

In [21]:
# Gtmp = pgv.AGraph('tmp.dot')
# G = nx.Graph(Gtmp)
# nx.draw(G)
# # plt.show()


$\mathcal{L}(y \vert x) = - \log\bigg\{\sum_{k=1}^K \pi_k(x)  \mathcal{N}\big(y \vert \mu_k(x), \Sigma_k(x)\big)\bigg\}$

# 7. Load and test
## 7.1 load and init

In [22]:
Model_name = "Conv_N_g=2"

model_path_MLP = "mlp_train=300_MLP.pth"
model_path_Conv = "mlp_train=300_Conv.pth"
model_path_init = "mlp_init.pth"
# mlp = Conv_1_1(opt.N_gaussians,kernel_size=12,stride=3)
mlp = Conv_1_4(opt.N_gaussians,kernel_size=9,stride=3)
# mlp = MLP_1_4(opt.N_gaussians)
model_data = torch.load(model_path_Conv)
mlp.load_state_dict(model_data)


mlp = mlp.to(device=device)

- 计算metric

In [23]:

win_pi_str = "pi distrb-"+Model_name
win_mu_str = "mu distrb-"+Model_name
win_sigma_str = "sigma distrb-"+Model_name

win_scale_str = "scale distrb-"+Model_name
win_shape_str = "shape distrb-"+Model_name

In [24]:
mlp.eval()
with torch.no_grad():
    #Compute the metric
    total_test_metric, GT_metric = validate(mlp,test_loader,opt.N_gaussians, opt.MIN_LOSS,device)
    draw_metric(0, total_test_metric.cpu(), GT_metric)
    print(f"========== IN Test dataset, NN Model:  {total_test_metric.detach().cpu().numpy()} ==========")
    print(f"========== IN Test dataset, the GTs: {GT_metric.detach().cpu().numpy()} ==========")
    print(f"========== The diff: {GT_metric.detach().cpu().numpy()[0,0] - total_test_metric.detach().cpu().numpy()} ==========")

    # Save pi and mu sigma
    cnt = 0
    pi = torch.tensor([])
    mu = torch.tensor([])
    sigma = torch.tensor([])
    for test_batch_id, test_data in enumerate(test_loader):
        test_input_data, test_target, _, test_setting , test_metric = test_data
        test_input_data = test_input_data.to(device)
        test_target = test_target.to(device)
        cnt = cnt + len(test_input_data)
        test_pi, test_mu, test_sigma = mlp(test_input_data)

        # draw_mdn_wei(test_pi[0,:].clone().detach(), test_mu[0,:].clone().detach(), test_sigma[0,:].clone().detach(), test_target[0].clone().detach(), test_input_data[0].clone().detach(),-(test_batch_id+100), opt.N_gaussians)
        # draw_mdn_wei(test_pi[1,:].clone().detach(), test_mu[1,:].clone().detach(), test_sigma[1,:].clone().detach(), test_target[1].clone().detach(), test_input_data[1].clone().detach(),-(test_batch_id+101), opt.N_gaussians)
        # draw_mdn_wei(test_pi[2,:].clone().detach(), test_mu[2,:].clone().detach(), test_sigma[2,:].clone().detach(), test_target[2].clone().detach(), test_input_data[2].clone().detach(),-(test_batch_id+102), opt.N_gaussians)
        # draw_mdn_wei(test_pi[3,:].clone().detach(), test_mu[3,:].clone().detach(), test_sigma[3,:].clone().detach(), test_target[3].clone().detach(), test_input_data[3].clone().detach(),-(test_batch_id+103), opt.N_gaussians)

        pi = torch.cat([pi,torch.squeeze(test_pi.detach().cpu())],dim=0)
        mu = torch.cat([mu,torch.squeeze(test_mu.detach().cpu())],dim=0)
        sigma = torch.cat([sigma,torch.squeeze(test_sigma.detach().cpu())],dim=0)



========== IN Test dataset, NN Model:  6.412590980529785 ==========
========== IN Test dataset, the GTs: [[ 6.857092 10.584564]] ==========
========== The diff: 0.4445009231567383 ==========


In [1864]:
def plot_scatter(viz,data, win_str, ylabel):
    for i in range(len(data)):
         # scatter的input shape：N*2或N*3的
        viz.scatter(X = np.array([[i],[data[i,0].numpy().item()]]).reshape(1,2), env="001",win=win_str, opts= dict(title=win_str, xlabel="idx", ylabel=ylabel),update="append")
        viz.scatter(X = np.array([[i],[data[i,1].numpy().item()]]).reshape(1,2), env="001",win=win_str, opts= dict(title=win_str, xlabel="idx", ylabel=ylabel),update="append")
        # viz.scatter(X = np.array([[i],[data[i,2].numpy().item()]]).reshape(1,2), env="001",win=win_str, opts= dict(title=win_str, xlabel="idx", ylabel=ylabel),update="append")

        # viz.scatter(X = np.array([[i],[data[i].numpy().item()]]).reshape(1,2), env="001",win=win_str, opts= dict(title=win_str, xlabel="idx", ylabel=ylabel),update="append")

    print("Done: "+win_str)

In [1865]:
# plot_scatter(viz,mu,win_mu_str,"mu")
# plot_scatter(viz,sigma,win_sigma_str,"sigma")
# plot_scatter(viz,pi,win_pi_str,"pi")

plot_scatter(viz,mu,win_scale_str,"scale")
plot_scatter(viz,sigma,win_shape_str,"shape")
plot_scatter(viz,pi,win_pi_str,"pi")

Done: scale distrb-Conv_N_g=2
Done: shape distrb-Conv_N_g=2
Done: pi distrb-Conv_N_g=2
